In [1]:
from textwrap import dedent
from tqdm import tqdm
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM


In [4]:
test = pd.read_csv('F:/Desktop/Shirlyne/kenya-clinicians/Data/test.csv')
test.sample(5)

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel
14,ID_OTEWX,uasin gishu,national referral hospitals,16.0,i am a nurse with 16 years of experience in ca...,adult health,critical care
27,ID_BMKRM,uasin gishu,health centres,27.0,i am a nurse with 27 years of experience in ge...,adult health,surgery
80,ID_PGEOO,kakamega,sub county hospitals and nursing homes,NaN,i am a nurse working in a sub county hospitals...,surgical care,internal medicine surgery
51,ID_MXHUP,uasin gishu,dispensaries and private clinics,22.0,i am a nurse with 22 years of experience in ge...,adult health,internal medicine
94,ID_MLUOW,uasin gishu,national referral hospitals,17.0,i am a nurse with 17 years of experience in ge...,child health,paediatrics


In [7]:
# Load model
model = OllamaLLM(model="qwen2.5:0.5b", max_tokens=250, temperature= 0.1,
                 num_ctx = 1024 ,#reduce memory usage, 
                 ).with_retry()

# Optimized prompt with enhanced structure and clinical tone
optimized_prompt_template = """
You are a Kenyan clinical officer. Given a medical scenario written by a nurse, your task is to:

1. Summarize the patient case using professional clinical language.
2. Provide a structured response with diagnosis, investigations, management, and follow-up.
3. Ensure the format and style matches the following examples exactly.

### Example 1:
I am a nurse with 18 years of experience in General nursing working in a Sub-county Hospitals and Nursing Homes in Uasin Gishu county in Kenya. A 4-year-old child presents to the emergency department with second-degree burns on the forearm after accidentally touching a hot stove. The child was playing in the kitchen when they reached out to touch the stove. The burns cover about 5% of the total body surface area. The child is alert and crying, with redness, blisters, and swelling on the affected area...
Questions:
1. What is the immediate treatment protocol for second-degree burns in paediatric patients?
2. Should any tetanus prophylaxis be considered in this case?
3. What follow-up care should be recommended for burn healing?
----------------------------------------------------------------------------------------------------
Summary:
A 4-year-old with 5% superficial burns. No other injuries.

Immediate Management:
 * Paracetamol analgesics to ensure child has minimal or no pain
 * Cleaning and dressing of wound with silver sulfadiazine
 * Topical prophylactic considered in this case

Follow-up Care:
 * Good nutrition – high-protein diet

**********************************

### Example 2:
I am a nurse with 17 years of experience in General nursing working in a National Referral Hospitals in Uasin Gishu county in Kenya. A 6-year-old girl presented to the emergency department with complaints of vomiting and abdominal pains. Patient is known diabetic on insulin and does not take as scheduled because of lack of funds.
On assessment the girl is confused, having Kussmaul breathing and fruity-scented breath. Has dry tongue and reports blurred vision. Temp 37°C, Pulse 120 bpm, Resp 48 bpm (rapid and labored), SpO2 90% on room air.
Questions:
What is the diagnosis of the patient?
What is the most immediate management?
What health education will be given to the patient and family?
Which investigations will be ordered?
----------------------------------------------------------------------------------------------------
Summary:
6-year-old with vomiting and abdominal pain. Known diabetic on insulin but non-adherent due to lack of funds. Confused, Kussmaul breathing, fruity breath, dry tongue, blurred vision.Temp (normal), Pulse ↑120, RR ↑48, SpO₂ ↓90%.

Diagnosis:
Diabetic Ketoacidosis (DKA) due to insulin omission in a type 1 diabetic patient.

Immediate Management:
 * Insert IV line and administer normal saline bolus
 * Continuous insulin infusion (0.1 U/kg/hr)
 * Monitor blood glucose
 * Add potassium to IV fluids if hypokalemic
 * Treat underlying infection
 * Monitor ketones, pH, and bicarbonate
 
 Investigations:
 * Urinalysis
 * Blood gas analysis
 * Random blood sugar (RBS)
 * HbA1c
 * UECs
 * CBC
**********************************

### INPUT:
{question}
### OUTPUT:
"""


prompt = ChatPromptTemplate.from_template(optimized_prompt_template)
chain = prompt | model



In [ ]:
test['Clinician'] = ""
for idx, row in tqdm(test.iterrows(), total =len(test)):
  question = row['Prompt']
  # display(Markdown(chain.invoke({"question": question})))
  answer = chain.invoke({"question": question})
  # print(processed_answer)
  processed_answer = answer.replace('\n\n', ' ').replace('\n', ' ').replace('**', '')
  test.loc[idx, 'Clinician'] = processed_answer

  0%|          | 0/100 [01:03<?, ?it/s]


NameError: name 'processed_answer' is not defined